In [ ]:
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def load_data1(ground_truth_file, prediction_file):
    ground_truth = {}
    with open(ground_truth_file, 'r') as f:
        for line in f:
            image_id_xmin, ymin, xmax, ymax, class_id = line.strip().split(',')
            image_id, xmin = image_id_xmin.split(' ')[0], image_id_xmin.split(' ')[1]
            if image_id not in ground_truth:
                ground_truth[image_id] = {}
            ground_truth[image_id] = {'class_id': class_id, 'xmin': float(xmin), 'ymin': float(ymin), 'xmax': float(xmax), 'ymax': float(ymax)}

    predictions = {}
    with open(prediction_file, 'r') as f:
        for line in f:
            image_id_xmin, ymin, xmax, ymax, class_id, confidence = line.strip().split(',')
            image_id, xmin = image_id_xmin.split(' ')[0], image_id_xmin.split(' ')[1]
            if image_id not in predictions:
                predictions[image_id] = {}
            predictions[image_id] = {'class_id': class_id, 'confidence': float(confidence), 'xmin': float(xmin), 'ymin': float(ymin), 'xmax': float(xmax), 'ymax': float(ymax)}

    return ground_truth, predictions


In [ ]:
ground_truth, predictions = load_data1('/content/drive/MyDrive/CS512/HW4/gt_label.txt', '/content/drive/MyDrive/CS512/HW4/  ')
predictions = {key: predictions[key] for key in sorted(predictions,key=lambda x:predictions[x]['confidence'], reverse=True)}

In [ ]:
def compute_iou1(ground_truth_box, predicted_box):
    xmin = max(ground_truth_box['xmin'], predicted_box['xmin'])
    ymin = max(ground_truth_box['ymin'], predicted_box['ymin'])
    xmax = min(ground_truth_box['xmax'], predicted_box['xmax'])
    ymax = min(ground_truth_box['ymax'], predicted_box['ymax'])
    if xmin >= xmax or ymin >= ymax:
        return 0.0
    intersection_area = (xmax - xmin) * (ymax - ymin)
    ground_truth_area = (ground_truth_box['xmax'] - ground_truth_box['xmin']) * (ground_truth_box['ymax'] - ground_truth_box['ymin'])
    predicted_area = (predicted_box['xmax'] - predicted_box['xmin']) * (predicted_box['ymax'] - predicted_box['ymin'])
    union_area = ground_truth_area + predicted_area - intersection_area
    iou = intersection_area / union_area
    return iou


In [ ]:
thresholds =[0.25, 0.50, 0.75, 0.95]
for THS in thresholds:
  ROC = []
  for cl in range(2):
    count = 0
    TP = []
    FP = []
    for key1 in predictions:
        if int(predictions[key1]['class_id']) == cl and int(ground_truth[key1]['class_id']) == int(predictions[key1]['class_id']):
          if compute_iou1(ground_truth[key1], predictions[key1]) >= THS:
            TP.append(1)
            FP.append(0)
            count += 1
          else:
            FP.append(1)
            TP.append(0)
    cTP = np.cumsum(TP)
    cFP = np.cumsum(FP)
    PR = cTP / (cTP + cFP + np.finfo(np.float32).eps)
    RC = cTP / count
    ROC.append(np.trapz(PR, RC))
  print(' mAP for thresholds ',THS, ' is: ',np.mean(ROC))

 mAP for thresholds  0.25  is:  0.9989029629110797
 mAP for thresholds  0.5  is:  0.987274401308144
 mAP for thresholds  0.75  is:  0.9024274274477981
 mAP for thresholds  0.95  is:  0.0529729625849314
